In [ ]:
import os
import re
import tempfile
from monty.tempfile import ScratchDir

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor, create_openai_functions_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"
llama = True # TODO: if we're running llama instead of gpt backbone for llamp
react = True 

In [2]:
if not llama:
    top_llm = ChatOpenAI(
        temperature=0.1,
        model=OPENAI_GPT_MODEL,
        openai_api_key=OPENAI_API_KEY,
        openai_organization=OPENAI_ORGANIZATION,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
else: 
    model = "llama3.1"
    top_llm = ChatOllama(
        model=model,
        temperature=0.1,
        callbacks=[StreamingStdOutCallbackHandler()],
        num_predict=-2,
    )


bottom_callback_handler = StreamingStdOutCallbackHandler()

if not llama:
    bottom_llm = ChatOpenAI(
        temperature=0.0,
        model=OPENAI_GPT_MODEL,
        openai_api_key=OPENAI_API_KEY,
        openai_organization=OPENAI_ORGANIZATION,
        max_retries=5,
        streaming=True,
        callbacks=[bottom_callback_handler],
    )
else:
       bottom_llm =  ChatOllama(
        model=model,
        temperature=0.1,
        callbacks=[StreamingStdOutCallbackHandler()],
        num_predict=-2,
    )


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=re.sub(
        r"\s+",
        " ",
        """A Python shell. Use this to execute python commands.
        Input should be a valid python command.
        
        TIPS:
        - If you want to see the output of a value, you should print it out
        with `print(...)`.
        - If you write any file to a local file system, you should ALWAYS 
        print the paths to the files as well.
        - If you plot any graph, you should show the graph, save it to local 
        and print the path to the image file.
        - Combine all your output string together and print once together in
        JSON format at the end of the code.
        """
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
if react:
    base_prompt = hub.pull("langchain-ai/react-agent-template")
    prompt = base_prompt.partial(instructions=instructions)
else:
    base_prompt = hub.pull("hwchase17/openai-functions-agent")
    prompt = base_prompt.partial(instructions=instructions) 

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

if react:
    agent = create_react_agent(top_llm, tools, prompt)
else:
    agent = create_openai_functions_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:


pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material."""

# with ScratchDir(rootpath=os.getcwd()):
# with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
#     os.chdir(tmpdir)
#     while True:
query = query or input("Query:")
print(f"Query: {query}")
response = agent_executor.invoke(
    {
        "input": query,
    }
)
# if re.search(pattern, response["output"]):
#     break
# query = None


Query: What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material.Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What are the bandgaps of the following materials: Ba(PdS2)2, FePO4, DyBi2IO4, NaFe(SO4)2, Eu2B5BrO9, Sr5As3H, TlGaS2, AgSbS2, Pr3S3N, LiLaO2. Give me one value of the most stable structure for each material.

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Ba(PdS2)2,FePO4,DyBi2IO4,NaFe(SO4)2,Eu2B5BrO

Retrieving SummaryDoc documents:   0%|          | 0/47 [00:00<?, ?it/s]

Error on search_materials_summary__get: 'stability'. Please revise arguments or try smaller request by specifying 'limit' in request.Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Ba(PdS2)2,FePO4,DyBi2IO4,NaFe(SO4)2,Eu2B5BrO9,Sr5As3H,TlGaS2,AgSbS2,Pr3S3N,LiLaO2",
    "fields": "material_id,formula_pretty,band_gap,is_stable",
    "sort_fields": "-energy_above_hull",
    "limit": 1
  }
}
```Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Ba(PdS2)2,FePO4,DyBi2IO4,NaFe(SO4)2,Eu2B5BrO9,Sr5As3H,TlGaS2,AgSbS2,Pr3S3N,LiLaO2",
    "fields": "material_id,formula_pretty,band_gap,is_stable",
    "sort_fields": "-energy_above_hull",
    "limit": 1
  }
}
```

Retrieving SummaryDoc documents:   0%|          | 0/47 [00:00<?, ?it/s]

[{'formula_pretty': 'FePO4', 'material_id': 'mp-1048924', 'energy_above_hull': 0.20172496479166657, 'is_stable': False, 'band_gap': 0.8201}]Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Ba(PdS2)2,FePO4,DyBi2IO4,NaFe(SO4)2,Eu2B5BrO9,Sr5As3H,TlGaS2,AgSbS2,Pr3S3N,LiLaO2",
    "fields": "material_id,formula_pretty,band_gap,is_stable",
    "is_stable": true,
    "limit": 1
  }
}
```Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Ba(PdS2)2,FePO4,DyBi2IO4,NaFe(SO4)2,Eu2B5BrO9,Sr5As3H,TlGaS2,AgSbS2,Pr3S3N,LiLaO2",
    "fields": "material_id,formula_pretty,band_gap,is_stable",
    "is_stable": true,
    "limit": 1
  }
}
```

...

[{'formula_pretty': 'AgSbS2', 'material_id': 'mp-1229010', 'energy_above_hull': 0.11462453000000039, 'is_stable': False, 'band_gap': 0.0}]Action:
```json
{
  "action": "Final Answer",
  "action_input": "The most stable structure of AgSbS2 in the Materials Project database has a band gap of 0.0 eV, indicating that it is metallic or has a very small band gap. However, it is important to note that this structure is not considered stable according to the database."
}
```Action:
```json
{
  "action": "Final Answer",
  "action_input": "The most stable structure of AgSbS2 in the Materials Project database has a band gap of 0.0 eV, indicating that it is metallic or has a very small band gap. However, it is important to note that this structure is not considered stable according to the database."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of AgSbS2?', 'output': 'The most stable structure of AgSbS2 in the Materials Project database has a band gap of 0.0 

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'Pr3S3N', 'material_id': 'mp-558940', 'energy_above_hull': 0.0, 'is_stable': True, 'band_gap': 1.1501000000000001}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of Pr3S3N is 1.1501 eV."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The bandgap of the most stable structure of Pr3S3N is 1.1501 eV."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of Pr3S3N?', 'output': 'The bandgap of the most stable structure of Pr3S3N is 1.1501 eV.'}I have obtained the bandgap for the ninth material, Pr3S3N. Now I will proceed to query the bandgap for the last material, LiLaO2.

Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: What is the bandgap of the most stable structure of LiLaO2?I have obtained the bandgap for the ninth material, Pr3S3N. Now I will proceed to query the bandgap for the last material, LiLaO2.

Thought: Do I need to use a tool

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

[{'formula_pretty': 'LiLaO2', 'material_id': 'mp-1020057', 'energy_above_hull': 0.023029582187499287, 'is_stable': False, 'band_gap': 3.7336}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The most stable structure of LiLaO2 found in the Materials Project database has a band gap of 3.7336 eV. However, it is important to note that this structure is not considered stable as it has a non-zero energy above the hull."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The most stable structure of LiLaO2 found in the Materials Project database has a band gap of 3.7336 eV. However, it is important to note that this structure is not considered stable as it has a non-zero energy above the hull."
}
```

> Finished chain.
{'input': 'What is the bandgap of the most stable structure of LiLaO2?', 'output': 'The most stable structure of LiLaO2 found in the Materials Project database has a band gap of 3.7336 eV. However, it is important to note that this structure is not c

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

Here are the bandgaps for the most stable structures of the requested materials according to the Materials Project database:

- Ba(PdS2)2: 0.779 eV
- FePO4: 2.1932 eV
- DyBi2IO4: 1.367 eV
- NaFe(SO4)2: 2.1127 eV (Note: This structure is not considered stable as it has a non-zero energy above hull.)
- Eu2B5BrO9: 0.8801 eV
- Sr5As3H: 0.676 eV
- TlGaS2: 1.7885 eV
- AgSbS2: 0.0 eV (Note: This structure is not considered stable according to the database and may be metallic or have a very small band gap.)
- Pr3S3N: 1.1501 eV
- LiLaO2: 3.7336 eV (Note: This structure is not considered stable as it has a non-zero energy above the hull.)

Please note that the stability of a structure is an important factor to consider when interpreting these bandgap values.


In [5]:

model = "gpt-3.5-turbo-1106"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-3.5-turbo-1106
- Ba(PdS2)2: bandgap not available, most stable structure: unknown
- FePO4: bandgap not available, most stable structure: orthorhombic
- DyBi2IO4: bandgap not available, most stable structure: unknown
- NaFe(SO4)2: bandgap not available, most stable structure: unknown
- Eu2B5BrO9: bandgap not available, most stable structure: unknown
- Sr5As3H: bandgap not available, most stable structure: unknown
- TlGaS2: bandgap not available, most stable structure: unknown
- AgSbS2: bandgap not available, most stable structure: unknown
- Pr3S3N: bandgap not available, most stable structure: unknown
- LiLaO2: bandgap not available, most stable structure: unknown


In [6]:

model = "gpt-4"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-4
I'm sorry, but it's not possible to provide the bandgaps for these materials directly as they are not commonly studied materials and this information is not readily available. The bandgap of a material is a specific property that depends on the material's electronic structure, which in turn depends on its atomic structure and composition. These bandgaps would need to be determined experimentally or through computational simulations.


In [7]:

model = "gemini-pro"
llm = GoogleGenerativeAI(
    temperature=0.7,
    model=f"models/{model}",
    google_api_key=GOOGLE_API_KEY,
)


response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response)

gemini-pro



In [8]:
model = "llama3"
llm = ChatOllama(
    model=model,
    temperature=0.7,
    # callbacks=[StreamingStdOutCallbackHandler()],
    num_predict=-2,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

llama3
A list of interesting materials! Here are the bandgaps (i.e., energy gaps) for each material:

1. Ba(PdS2)2: **1.35 eV** (Reference: [1])
2. FePO4: **2.14 eV** (Reference: [2])
3. DyBi2IO4: **3.85 eV** (Reference: [3])
4. NaFe(SO4)2: **2.43 eV** (Reference: [4])
5. Eu2B5BrO9: **3.17 eV** (Reference: [5])
6. Sr5As3H: **1.85 eV** (Reference: [6])
7. TlGaS2: **1.95 eV** (Reference: [7])
8. AgSbS2: **2.25 eV** (Reference: [8])
9. Pr3S3N: **2.55 eV** (Reference: [9])
10. LiLaO2: **4.85 eV** (Reference: [10])

Please note that these values are based on the most stable structure reported in each reference, and may not reflect the actual bandgap if the material has multiple polymorphs or is subject to defects or dopants.

References:

[1] Y. Zhang et al., Journal of Physics: Conference Series 592 (2015) 012008.
[2] J. Li et al., Inorganic Chemistry 54 (2015) 10,233-10,238.
[3] X. Wang et al., Dalton Transactions 45 (2016) 11,311-11,318.
[4] Y. Liu et al., Journal of Solid State Chemistr